In [2]:
import streamlit as st
import plotly.graph_objects as go

In [3]:
import pandas as pd
import altair as alt
from databricks import sql

In [5]:
# ---------------------------
# 1. Databricks Connection
# ---------------------------
# Replace with your credentials or set them in Streamlit secrets
DATABRICKS_SERVER = st.secrets["databricks"]["server_hostname"]
DATABRICKS_HTTP_PATH = st.secrets["databricks"]["http_path"]
DATABRICKS_TOKEN = st.secrets["databricks"]["token"]

In [6]:
def query_databricks(sql_query):
    with sql.connect(
        server_hostname=DATABRICKS_SERVER,
        http_path=DATABRICKS_HTTP_PATH,
        access_token=DATABRICKS_TOKEN
    ) as connection:
        with connection.cursor() as cursor:
            cursor.execute(sql_query)
            result = cursor.fetchall()
            columns = [desc[0] for desc in cursor.description]
            return pd.DataFrame(result, columns=columns)


In [7]:
# ---------------------------
# 2. App Header & Filters
# ---------------------------
st.set_page_config(page_title="E-commerce Analytics Dashboard", layout="wide")
st.title("E-commerce Analytics Dashboard")

# Example: Device filter
device_filter = st.multiselect(
    "Select Device Type(s):",
    ["Desktop", "Mobile", "Tablet"],
    default=["Desktop", "Mobile", "Tablet"]
)

2025-08-26 17:49:19.920 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-26 17:49:19.924 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-26 17:49:20.437 
  command:

    streamlit run c:\Users\SarigaMadathil\OneDrive - ROKO Labs LLC\Projects\Growth Analytics\.venv\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-08-26 17:49:20.439 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-26 17:49:20.441 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-26 17:49:20.443 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-26 17:49:20.447 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-26 17:49:20.448 Thread 'MainThread': missing Scr

In [8]:
# ---------------------------
# 3. Key Metrics (Cart Abandonment & Revenue)
# ---------------------------
st.header("Key Metrics")

# SQL queries (replace with your actual table names)
cart_query = "SELECT cart_abandonment_rate FROM cart_abandonment LIMIT 1"
revenue_query = "SELECT total_revenue, average_order_value FROM Revenue LIMIT 1"

cart_df = query_databricks(cart_query)
revenue_df = query_databricks(revenue_query)

cart_abandonment_rate = cart_df["cart_abandonment_rate"].iloc[0]
total_revenue = revenue_df["total_revenue"].iloc[0]
average_order_value = revenue_df["average_order_value"].iloc[0]

col1, col2, col3 = st.columns(3)
col1.metric("Cart Abandonment Rate", f"{cart_abandonment_rate*100:.1f}%")
col2.metric("Total Revenue", f"${total_revenue:,.0f}")
col3.metric("Average Order Value", f"${average_order_value:.2f}")


2025-08-26 18:03:39.110 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-26 18:03:39.111 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-26 18:03:39.111 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-26 18:04:04.518 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-26 18:04:04.520 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-26 18:04:04.521 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-26 18:04:04.522 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-26 18:04:04.527 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

DeltaGenerator()

In [9]:
# ---------------------------
# 4. Device Type Performance
# ---------------------------
st.header("Device Type Performance")

device_query = """
SELECT device_type, purchases, sessions, conversion_rate
FROM device_types
"""
device_df = query_databricks(device_query)
filtered_df = device_df[device_df["device_type"].isin(device_filter)]

device_chart = alt.Chart(filtered_df).mark_bar().encode(
    x="device_type",
    y="purchases",
    color="device_type",
    tooltip=["sessions", "purchases", "conversion_rate"]
).properties(width=500)

st.altair_chart(device_chart, use_container_width=True)


2025-08-26 18:06:59.977 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-26 18:06:59.980 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-26 18:06:59.984 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
c:\Users\SarigaMadathil\OneDrive - ROKO Labs LLC\Projects\Growth Analytics\.venv\Lib\site-packages\altair\utils\core.py:264: UserWarning: I don't know how to infer vegalite type from 'empty'.  Defaulting to nominal.
  warnings.warn(
c:\Users\SarigaMadathil\OneDrive - ROKO Labs LLC\Projects\Growth Analytics\.venv\Lib\site-packages\altair\utils\core.py:264: UserWarning: I don't know how to infer vegalite type from 'empty'.  Defaulting to nominal.
  warnings.warn(
2025-08-26 18:07:02.557 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-26 18:07:02.557 Thread

DeltaGenerator()

In [10]:
# ---------------------------
# 5. Funnel Analysis
# ---------------------------
st.header("Funnel Analysis")

funnel_query = """
SELECT website_visits, product_views, add_to_cart, purchases, conversion_rate, drop_off, drop_off_rate
FROM funnel_analysis
LIMIT 1
"""
funnel_df = query_databricks(funnel_query).T.reset_index()
funnel_df.columns = ["stage", "value"]

funnel_chart = alt.Chart(funnel_df).mark_bar().encode(
    x="stage",
    y="value",
    color="stage",
    tooltip=["value"]
).properties(width=600)

st.altair_chart(funnel_chart, use_container_width=True)


2025-08-26 18:07:17.920 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-26 18:07:17.923 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-26 18:07:17.926 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-26 18:07:20.006 Serialization of dataframe to Arrow table was unsuccessful. Applying automatic fixes for column types to make the dataframe Arrow-compatible.
Traceback (most recent call last):
  File "c:\Users\SarigaMadathil\OneDrive - ROKO Labs LLC\Projects\Growth Analytics\.venv\Lib\site-packages\streamlit\dataframe_util.py", line 821, in convert_pandas_df_to_arrow_bytes
    table = pa.Table.from_pandas(df)
            ^^^^^^^^^^^^^^^^^^^^^^^^
  File "pyarrow\\table.pxi", line 4793, in pyarrow.lib.Table.from_pandas
  File "c:\Users\SarigaMadathil\OneDrive - ROKO Labs LLC\Projects\Growth Analytics\.venv\Li

DeltaGenerator()

In [11]:
# ---------------------------
# 6. Exit & Session Insights
# ---------------------------
st.header("Exit & Session Insights")

exit_query = """
SELECT purchase, avg_bounce_rates, avg_exit_rates
FROM exit_rates
"""
exit_df = query_databricks(exit_query)

exit_chart = alt.Chart(exit_df).mark_circle(size=200).encode(
    x="avg_bounce_rates",
    y="avg_exit_rates",
    color="purchase",
    tooltip=["avg_bounce_rates", "avg_exit_rates"]
)

st.altair_chart(exit_chart, use_container_width=True)

2025-08-26 18:07:38.441 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-26 18:07:38.445 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-26 18:07:38.448 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-26 18:07:40.250 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-26 18:07:40.251 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-26 18:07:40.253 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


DeltaGenerator()